In [11]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import keras.layers as L
import keras.models as M
import os
from sklearn.metrics import *
from livelossplot import PlotLossesKeras
from keras.models import load_model
import tensorflow as tf

In [2]:
inputdf = pd.read_csv("../New Data/inputdf.csv")
outputdf = pd.read_csv("../New Data/outputdf.csv")

In [ ]:
l = []
for i in range(6, 568, 14):
    if(i == 482):
        break
    if(i not in [482, 483]):
        print(i, " - ", inputdf.columns[i])
    l.append(i)
i += 2
for t in range(i, inputdf.shape[1] - 6, 14):
    print(t, " - ", inputdf.columns[t])
    l.append(t)
i-=2
for t in range(i, inputdf.shape[1]):
    print(t, " - ", inputdf.columns[t])
    l.append(t)

In [4]:
inp = L.Input(shape=(9,))
x = L.Dense(1024, activation='relu')(inp)
x = L.Dropout(0.2)(x)
x = L.Dense(1024, activation='relu')(x)
x = L.Dropout(0.2)(x)
x = L.Dense(1024, activation='relu')(x)
x = L.Dropout(0.2)(x)
x = L.Dense(1024, activation='relu')(x)
x = L.Dropout(0.2)(x)
x = L.Dense(1024, activation='relu')(x)
x = L.Dropout(0.2)(x)
out = L.Dense(1, activation='relu')(x)
out = L.Dropout(0.2)(out)

model = M.Model(inputs=[inp],outputs=[out])
model.compile(optimizer='adam',loss=tf.losses.huber_loss,metrics=['mae'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 9)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              10240     
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dropout_3 (Dropout)          (None, 1024)              0         
__________

In [ ]:
#writer = pd.ExcelWriter('StaticANNStage1.xlsx')
d = pd.DataFrame()
split = int(0.8*inputdf.shape[0])
for food in range(0,1):
    if(food != 6):
        #d = pd.DataFrame()
        print ("\n\nFood : ",outputdf.columns[food])
        print ("-------------------------------------------------------------")
        
        col_list = [6,20,34,48,426]
        
        xtrain , xtest = inputdf.values[:split,list(range(2,6))] , inputdf.values[split:,list(range(2,6))]
        ytrain , ytest = outputdf.values[:split,food].reshape((-1,1)) , outputdf.values[split:,food].reshape((-1,1))
        for j in col_list:
            xtrain , xtest = np.hstack((xtrain,inputdf.values[:split,food + j].reshape((-1,1)))) , np.hstack((xtest,inputdf.values[split:,food+j].reshape((-1,1))))
        
        print (xtrain.shape,ytrain.shape,xtest.shape,ytest.shape)
        model.fit(xtrain,ytrain,epochs=100,batch_size=32)
        ypred = model.predict(xtest,batch_size=32)
        d['Date'] = pd.Series(inputdf.iloc[split:,0])
        d['yPred'] = ypred
        d['yTest'] = ytest
        d['mape'] = np.abs(ypred.reshape((-1,1)) - ytest ) / ytest * 100
#        d.to_excel(writer,sheet_name=outputdf.columns[food],index=False)
#writer.save()